In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.functions import col

# Set the location of the Delta Lake and Kafka packages
delta_package = "io.delta:delta-spark_2.12:3.0.0"  # Replace with the correct Delta version
kafka_package = "org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1"  # Replace with the correct Spark version
xml_package = "com.databricks:spark-xml_2.12:0.14.0"

# Initialize Spark Session with Delta Lake and Kafka support
spark = SparkSession.builder \
    .appName("test") \
    .config("spark.jars.packages", f"{delta_package},{xml_package}") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

:: loading settings :: url = jar:file:/home/spark/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/spark/.ivy2/cache
The jars for the packages stored in: /home/spark/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
com.databricks#spark-xml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f27efca6-9aa5-4424-86c7-a6e9c9a0ae68;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.0.0 in central
	found io.delta#delta-storage;3.0.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found com.databricks#spark-xml_2.12;0.14.0 in central
	found commons-io#commons-io;2.8.0 in central
	found org.glassfish.jaxb#txw2;2.3.4 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.2.5 in central
:: resolution report :: resolve 141ms :: artifacts dl 4ms
	:: modules in use:
	com.databricks#spark-xml_2.12;0.14.0 from central in [default]
	commons-io#commons-io;2.8.0 from central in [default]
	io.delta#delta-spark_2.12;3.0.0 from central in [default]
	io.delta#delta-storage;3.0.0 from central in [defa

In [2]:
raw_df = spark.read.format('parquet').option("mergeSchema", "true").load('hdfs://spark-test1:9000/raw/transactions')
raw_df_20 = raw_df.orderBy(col("_raw_insert_timestamp").desc()).limit(20)
print(raw_df.count())

299


In [6]:
processed_df = spark.read.format('parquet').option("mergeSchema", "true").load('hdfs://spark-test1:9000/dlq/processed/transactions_failed_batches')
processed_df_20 = processed_df.orderBy(col("_processed_insert_timestamp").desc()).limit(20)
display(processed_df_20.toPandas())

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `_processed_insert_timestamp` cannot be resolved. Did you mean one of the following? [`_error_insert_timestamp`, `_raw_insert_timestamp`, `_error_insert_date`, `_raw_insert_date`, `_error_insert_hour`].;
'Sort ['_processed_insert_timestamp DESC NULLS LAST], true
+- Relation [key#40,value#41,topic#42,partition#43,offset#44,timestamp#45,timestampType#46,_raw_insert_timestamp#47,_raw_insert_date#48,_raw_insert_hour#49,error_message#50,_traceback#51,_batch_id#52,_error_insert_timestamp#53,_error_insert_date#54,_error_insert_hour#55] parquet


In [10]:
processed_df = spark.read.format('parquet').option("mergeSchema", "true").load('hdfs://spark-test1:9000/dlq/processed/transactions_failed_batches')
processed_df_20 = processed_df.orderBy(col("_error_insert_timestamp").desc()).limit(20)
#display(processed_df_20.toPandas())
for row in processed_df_20.collect():
    print(row['error_message'])
    break

[UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `_corrupt_file` cannot be resolved. Did you mean one of the following? [`topic`, `offset`, `partition`, `json_data`, `parsed`].;
'Filter isnull('_corrupt_file)
+- Project [key#0, value#1, topic#2, partition#3, offset#4, timestamp#5, timestampType#6, _raw_insert_timestamp#7, _raw_insert_date#8, _raw_insert_hour#9, json_data#86, decoded_content#98, xml_content#112, xmldatatocatalyst(cast(xml_content#112 as string), StructField(Transaction,ArrayType(StructType(StructField(TransactionId,LongType,true),StructField(Amount,FloatType,true),StructField(CustomerId,LongType,true),StructField(DateTime,TimestampType,true),StructField(Location,StringType,true),StructField(Result,StringType,true)),true),true), com.databricks.spark.xml.XmlOptions@19c6d002) AS parsed#126]
   +- Project [key#0, value#1, topic#2, partition#3, offset#4, timestamp#5, timestampType#6, _raw_insert_timestamp#7, _raw_insert_date#8, _raw_insert_hour#9,